# 모델 불러오기

In [ ]:
# !pip install transformers==4.43.2
!pip install bitsandbytes==0.43.3
!pip install -U bitsandbytes
!pip install accelerate==0.33.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.3
    Uninstalling bitsandbytes-0.43.3:
      Successfully uninstalled bitsandbytes-0.43.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

# HF_TOKEN = os.getenv("HF_TOKEN")
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
def load_model(base_model, torch_dtype=torch.bfloat16, use_quantization=False, quantization_config=None):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    attn_implementation = "eager"

    if use_quantization:
        if quantization_config is None:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch_dtype,
                bnb_4bit_use_double_quant=True,
            )
    else:
        bnb_config = None

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch_dtype,
        device_map="auto",
        attn_implementation=attn_implementation
    )

    if not use_quantization:
        model.to(device)

    return model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
device

'cuda'

In [ ]:
model = load_model(model_name, use_quantization=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = model.config


print(f"Memory Allocated: {round(torch.cuda.memory_allocated() / (1024 ** 2), 3)}GB")
print(f"Memory Reserved: {round(torch.cuda.memory_reserved() / (1024 ** 2), 3)}GB")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Memory Allocated: 5438.369GB
Memory Reserved: 5488.0GB


# 예시 작성하기

In [ ]:
example = """
'''
이 제품은 가격대비 가성비가 좋고, 토핑도 풍부해서 맛있다는 소비자들의 평이 많아요. 집에서 간편하게 끓여 먹을 수 있어서 편리하고, 양도 푸짐해서 가족 모두에게 적합해요. 다만, 유통기한이 짧아서 버리는 경우가 있었다는 의견도 있어요.
'''

'''
가격 대비 만족스러운 제품이에요. 순두부찌개나 차돌박이와 함께 먹으면 더 맛있어요. 밀키트의 장점을 모두 갖춘 제품으로 편리해요. 고기가 푸짐하고 씹는 재미도 있어서 두 명이 함께 먹기 좋아요. 신선한 재료로 만들어져 있어 매번 자주 구매하는 제품입니다. 백반집 수준의 퀄리티에 매우 만족하실 거예요.
'''

'''
가격 대비로 만족하고, 집에서 간편하게 전문점 맛을 느낄 수 있어요. 청국장이 특히 좋다는 의견도 많아요. 양도 푸짐하고, 냄새도 거의 없어서 신선한 재료를 사용한 것 같아요.
'''

'''
신선하고 좋은 제품이라고 하셔서 만족스러우시다면, 가성비가 좋아서 자주 구매하시는 고객님들이 많으실 것 같아요. 집에 추가 재료를 넣어 맛을 더해도 좋다는 팁도 있어요. 대학생 아이들이 혼자 먹기에도 편리하다는 이야기도 있네요. 스팸과 김치를 함께 끓여먹으면 더 맛있다는 소중한 정보도 함께 전해드릴게요!
'''

'''
요리하기 편하고 빠르게 먹을 수 있는 제품이에요. 냄비를 바로 끓일 수 있어서 편리하고, 12인용으로 적당한 양이에요. 혼자 먹기에도 적당하고, 맛있어서 가게에서 사먹는 것 같아요. 맵지 않아서 부드럽고, 부재료를 더 넣어서 더 맛있게 먹을 수 있어요. 만족스러운 제품이라 재구매 의사가 있습니다!
'''
"""

# 데이터 적용하기(상품별, 긍부정별 리뷰 요약)

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Set pandas options to display full content of DataFrame
pd.set_option('display.max_colwidth', None)  # No truncation of column width
pd.set_option('display.max_rows', None)      # No truncation of rows
pd.set_option('display.max_columns', None)   # No truncation of columns

origin1 = pd.read_csv("/content/drive/MyDrive/50개씩_감정태깅(최종).csv")
origin1.head()

Mounted at /content/drive


,product_id,product_type,date,help_num,cleaned_Kiwi_review,sentiment
0,5013198,한우 도가니탕,2023-11-30,2,하트찜꽁 쫀득한 도가니탕 맛있네요 국 간은 싱거운데 소금 간 하면 딱이에요,positive
1,5013198,한우 도가니탕,2023-09-25,2,일부러 내용 청 가물을 보고 샀습니다 가장 청 가물과 나트륨이 적었구요 가격은 음 착하지는 않습니다 추가 구매해서 사 먹을 맛은 아닌 걸로 제가 도가니 매니아가 아니라 그런 걸까요 아니면 후기를 좋은 말만 쓰면 뭐가 있는 건가요 컬리 초보자라 궁금하군요,negative
2,5013198,한우 도가니탕,2023-01-24,2,기름 너무 많아서 느끼함,negative
3,5013198,한우 도가니탕,2024-06-03,1,믿고 먹는 설성 목장이예요 무 항생제인데 도가니도 꽤 들어 있고 가격 대비 좋네요,positive
4,5013198,한우 도가니탕,2024-03-05,1,설성 목장은 소금과 첨가물 안 들어 있어서 좋아요,positive


In [ ]:
def extract_assistant_text(outputs):
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("assistant")[-1].strip()

## Test 코드

In [ ]:
# 테스트할 product_id를 지정
test_product_id = 5048746  # 테스트할 product_id로 변경

# 해당 product_id의 데이터를 필터링
filtered_data = origin1[origin1['product_id'] == test_product_id]

# sentiment별 리뷰 개수 및 전체 리뷰 개수를 계산(test)
sentiment_counts = filtered_data.groupby(['product_id', 'product_type', 'sentiment']).size().reset_index(name='sentiment_count')
total_reviews = filtered_data.groupby(['product_id', 'product_type']).size().reset_index(name='total_reviews')

# sentiment 비율을 계산
sentiment_counts = sentiment_counts.merge(total_reviews, on=['product_id', 'product_type'])
sentiment_counts['sentiment_ratio'] = round((sentiment_counts['sentiment_count'] / sentiment_counts['total_reviews']) * 100)

# 비율을 계산한 후 리뷰 결합(test)
grouped_reviews = filtered_data.groupby(['product_id', 'product_type', 'sentiment'])['cleaned_Kiwi_review'].apply(' '.join).reset_index()

# 'neutral' 감정을 제외
grouped_reviews = grouped_reviews[grouped_reviews['sentiment'] != 'neutral']

# 결합된 리뷰와 비율을 병합
grouped_data = pd.merge(grouped_reviews, sentiment_counts[['product_id', 'product_type', 'sentiment', 'sentiment_ratio']],
                      on=['product_id', 'product_type', 'sentiment'])

# 결과 출력
grouped_data


,product_id,product_type,sentiment,cleaned_Kiwi_review,sentiment_ratio
0,5048746,청국장,negative,진한 맛일 줄 알았는데 생각보다 안 진하고 가벼운 맛이에요 단 거 좋아하는 제 입맛에도 달아서 부모님은 별로 안 좋아하셨어요 청국장이 맛있다는 평이 있어서 주문했는데 청국장 맛이 좀 부족했어요 청국장을 진하게 먹고 싶으신 분들은 실망할 수 있어요 컬리나우로 어제 13일 두 개를 배송시켜서 바로 냉동실에 넣고 오늘 해동을 시키는데 하나가 불량 이네요 고객 센터에 전화를 하니 택배로 보내라네요 박스에 박스도 없고 박스 구하러 동네 돌아다녀야할까요 그리고 구멍 난 국물 있는 상품인데 제가 컬리나우에서 사람이 회수해 가시면 안 되냐 하니 안 된다네요 좀 많이 황당하네요\n근데 이 상품 예전에도 불량이 있어서 교환했던 상품입니다 야채 흐물거리고 맛없는 데 왜 후기가 좋은지 모르겠음 음 제 취향은 아닌 듯 된장찌개 맛있게 먹어서 청국장도 구매했는데요\n청국장 맛이 약해서 아쉬워요\n재구매 의사는 없어요,12.0
1,5048746,청국장,positive,너어무 너무 진하고 맛있어요\n청국장 감별사입니다 왜냐하면 제가 청국장 러버거든요 시골 맛집 가면 청국장 먹을 지 비지 먹을지 좀 고민하지만 아무튼 컬리에서 비지는 좀 사 먹어 봤는데 청국장은 사 먹을 생각을 못하다 세 일에 떠서 샀던 것 같은데요\n\n결론 두 개 사서 잘했다 입니다\n컬리두부 반모만 더 넣어서 온 리 끓이기만했어요\n그런데 정말 진하고 맛있어요\n\n시골에서 청국장 따로 공수해 와 그 진한 청국장에 길들여진 입맛인데 이건 정말 만족했어요 어무니께서는 그래도 다르다고 하려나 모르겠으나 ㅎ 제 입에는 아니 무조건 무조건 재 구매할 거에요 엄마가 끓여 주던 고기 안 들어간 구수한 청국장이 먹고 싶어 주문해 봤어요\n한국인은 발효된 청국장 특유의 강한 향이 주기적으로 생각 나는 거 같아요 \n\n엄마 밥상 청국장은 향이 강하진 않지만 구수한 맛이 좋네요\n된장이랑 비슷하게 구수한 맛이 있지만 더 깊고 진한 맛이 나는 거 같아요\n콩의 고소한 맛이 기본적으로 깔려 있고 발효 과정에서 더욱 깊어진 고소함이 듬뿍 느껴져요\n\n그리고 무엇보다 고기 없이 두부 무 애호박 대파만 들어가서 담백해요\n여기에 순두부나 버섯 있으면 추가해서 바글바글 끓여 먹으면 밥 한 그릇 뚝딱이네요\n\n건강한 한 끼로 좋은 거 같아요 소고기 앞 다리 살 300그램 양파 볶아 주다가 청국장 넣고 끓이면서 애호박 조금 더 추가했어요 근데 진짜 너무 맛있게 먹어서 앞으로 마켓컬리 최애 될 듯 ㅠㅜㅠ 여기저기 추천했습니당 넘넘 맛있어 영 두부랑 배추만 좋아해서 조금 추가해서 막었 슺니 당 원래 두부 호박 무우 들어 있고 좋네옹 맛있어서 재구매했습니다 두부 호박 넣어서 끓이니 더 맛있어요 맛있어요 냄새도 적고 다른 채소 넣고 두부 넣고 끓여도 참 맛있네요 무더운 여름에도 가족 식사는 살뜰이 챙기는데\n정작 내 입맛만 못 챙겨서\n컬리 주문할 때 밥도둑 일만 한 걸 찾다가 \n청국장이 제격일 것 같아 구매\n일단은 청국장 끓여도 냄새가 심하지 않아 좋았구요 \n1인분 같은데 2명이 먹어도 넉넉할 것 같은 양과\n내용물에 두부가 생각보다 많았고 맛있었어요 \n그리고 애호박 파 무도 들어 있었는데\n식감이 청국장 국물과 너무 잘 맞네요\n집에 있는 부추 매운 고추도 넣어 입맛을 \n찾으려는 의지를 보이고\n밥 위에 열무를 썰어서 청국장과 먹었는데\n아 처음 한 입 먹고 계속 먹게 되는 구수하면서\n깔끔하게 담백한 맛 그리고 청국장 콩들도\n동글 동글 예쁘고 꽤 들어 있어서 다 마음에 \n들었어요\n종종 구매해야겠어요 \n나만의 입맛을 찾는 보양식이 여기 있었네요\n다음에는 잘 익은 김치를 청국장에 넣어 먹으려구요 \n벌써부터 기대됩니다 후기 좋아서 구매했는데 맛있어요 두부를 추가해서 먹었는데\n청국장 냄새 많이 안 나고 구수하니 맛있었어요 엄마 밥상 청국장이 젤 맛나요 물 더 넣고 감자 고추 고춧가루 더 넣어 먹음 요 ㅡ마땅한 반찬이나 국 없을 땐 좋은 듯 냉동이라 식감이 떨어지는 건 어쩔 수 없 컬리 상품 중 항상 구매하는 상품입니다\n말할 것도 더할 것도 없는\n베스트 강추 청국장이네요 짜지 않고 구수한 맛이 좋아 자주 구매해요\n다만 국내산 제품이 아니라 조금 아쉬워요 부모님 사 드렸는데\n맛있다고 하시네요\n간편하게 먹기 너무 좋은 거 같아요 엄마 밥상 청국장은 후기가 무려 3 만이 넘더라고요\n처음에 한두 번 사 먹을 때는 몰랐는데\n집에서 만든 청국장으로는 이 맛이 안 나더라고요\n구수한 맛이 생각날 때쯤 주문해 주는 청국장이에요\n\n기본으로 애호박 파 두부 등이 들어가 있어요\n추가 채소가 없더라도 600g 이니 2인분으로는 충분한 양이에요 \n저는 2 인 밥상 푸짐하게 즐기고 싶어서\n애호박 13 두 부 12 추가해서 먹어요\n\n먹기 전 냉장 실에서는 34시간 전에 \n상온에는 1시간 정도 해동하고\n바로 냄비에 붓고 끓이기만하면 돼요\n\n엄마 밥상 청국장을 끓이면 바로 우와 맛있다가 나오는 우리네 밥상 \n역시 대기업의 맛은 제가 따라갈 수 없나 봅니다\n컬리에서 청국장만도 여러 개 구매해 봤는데\n재구매하게 되는 청국장은 많지 않았어요 계속 다른 제품 시도해 보는 중 ㅎ\n그런데 이렇게 밀키트로 나오는 엄마 밥상은 \n이 청국장만의 맛이 생각나서 자주 찾아올 거 같습니다 ㅡ 청국장은 참 맛나네요 두브 추가 필수 찌개라기엔 국물이 많지만 맛있어요 두부 더 넣고 끓였더니 더 맛있어요 식당 안갸도 되겠네요 맛나요 두부 채소가 듬뿍 간도 적당하고 아주 맛나요 재구 매각입니다 맛있어서 재구매했어요 건더기도 적당히 들었어요 신랑이 워낙 청국장을 좋아해서 전에도 1번 구매했던 적이 있는 제품입니다 ㅎ그래서 믿고 구매했습니다\n냉동 상태로 배송이 오는 제품이고 먹기 전날 냉장고로 옮겨 냉장해 동을 해 줍니다\n바로 먹고 싶으면 유수해 동을 해 주어도 됩니다\n잘 해동이 된 청국장을 냄비에 옮겨서\n중불로 보글보글해질 때까지 조리를 해 줍니다\n조리할 때 두부를 조금 더 썰어 넣어도 되고 야채를 더 넣어도 되지만 저는 그냥 조리해도 괜찮아서\n그 상태 그대로 조리해서 먹었습니다\n청국장에 구수한 맛 감칠 맛이 있어 맛있게 먹었고\n딱 한 끼 식사용으로 좋습니다\n앞으로도 종종 구매 할 것 같아요 청국장은 엄마 밥상 꺼 매번 시켜 먹어요 후기 믿고 주문했어요 리뷰들이 너무 좋아서 담지 않을 수가 없었네요 맛 잇어요 \n반찬 없을 때 대비해 쟁엿어요 \n냉동으로 오는 거라 보관해 둿다 먹기도 좋네요 정말 칭찬해요\n두부 식감 좋구요\n짜지 않고 가성비 좋구요\n호박 양파 두부 좀 더 첨가해서 드셔도 좋아요\n기성 식품 맛 안 나요\n자주 구매할 거 같아요 냄새 가볍고 간이 가벼워서\n편하게 먹게 되요\n냉동실 쟁여템 이거 물건입니다 1 인분으로 딱 이 청국장이 제일 맛있어요 간도 세지 않고 구수하고 맛있어요 저는 두부 추가로 더 넣어서 먹었고요 들어 있는 두부는 냉동이라 어쩔 수 없이 약간의 스펀지화된 식감이지만 까칠한 저희 엄마도 잘 드셨어요 가성비 넘 좋으네요 간편해서 넘넘 좋아요 언제 막어도 나무 맛 일어요 가격도 맛도 좋아 여러 번 재구매했어요 청국장 남편이 좋아해서 가끔 주문합니다 \n들어 있는 딱딱한 두부만 빼고는 맛 좋고 괜찮습니다 김치 넣고 끓이면 얼큰하고 맛있어요 두부만 첨가하면 짜지 않고 맛있어요 청국장 두부를 추가로 넣어서 먹으니 양이 적당해요 냉동실 쟁여템 많이 짜지 않고 건더기도 실하고 냄새도 많이 나지 않아 너무 맛나요 내용물이 좋아요 재구매 의사 있음 잘 먹겠습니다 이것 저 것 넣어서 먹으려구용 아직 먹기 전인데 후기 너무 좋아 구매해 봤어요 평소 냄새 좀 나도 맛난 청국장이라면 좋아하는데 두부 추가해 맛나게 먹겠습니다 가끔 주문하는 제품이예요 맛이 깔끔해요 저희 가족들이 청국장을 너무 좋아하는데\n왜 그 동안 청국장은 주문할 생각을 못 한 걸까요\n\n오늘 점심으루 먹으려구 어젯밤에 주문했는데\n신랑한테 두부랑 대파 더 넣구 고춧가루두 좀 넣어서\n바글바글 끓여서 아들이랑 먹으랬 더니\n그대루 따라 하면서 끓였더라구요

In [ ]:
summary_data = []

# Loop through the grouped data (which is already filtered by a single product_id)
for _, row in grouped_data.iterrows():
    product_id = row['product_id']
    product_type = row['product_type']
    sentiment = row['sentiment']
    sentiment_ratio = row['sentiment_ratio']
    combined_reviews = row['cleaned_Kiwi_review']

    # Adjust the instruction based on sentiment
    if sentiment == 'positive':
        sentiment_instructions = "장점과 좋은 점에 집중해서 요약해주세요."
    elif sentiment == 'negative':
        sentiment_instructions = "불만 사항과 문제점에 집중해서 요약해주세요."

    # Prepare the initial summarization prompt
    summary_prompt = f"""
    당신은 상품에 대한 고객의 리뷰를 요약해서 소비자에게 정보를 제공하는 소비자 분석가입니다.

    **지시사항:**
    리뷰: {combined_reviews}\n
    이 리뷰들을 {sentiment_instructions}
    많이 나오는 내용을 중심으로 요약해주세요.
    요약한 문장을 한 번 더 매끄러운 문장으로 만들어 주세요.

    **제한사항:**
    브랜드 이름은 넣지 마세요.
    '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.

    **예시:**
    {example}

    **요약:**
    """
    inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{summary_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{combined_reviews}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # First pass: Generate the initial summary
    outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=200,
                             pad_token_id=tokenizer.eos_token_id,
                             repetition_penalty=1.0,
                             top_p = 0.90,
                             temperature=0.2
                             )

    initial_summary = extract_assistant_text(outputs)

    # Second pass: Condense the summary into one sentence
    rephrase_prompt = f"""
    당신은 문장을 간결하게 만드는 요약 전문가입니다.

    **지시사항:**
    initial summary : {initial_summary}
    위 문장을 요약해주세요.
    '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.:
    말투는 존댓말이어야 합니다.

    **요약:**
    """
    rephrase_inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{rephrase_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{initial_summary}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # Generate the final rephrased summary
    rephrase_outputs = model.generate(**tokenizer(rephrase_inputs, return_tensors="pt").to(device), max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    final_summary = extract_assistant_text(rephrase_outputs)

    # Save the final summary along with sentiment
    summary_data.append([test_product_id, product_type, sentiment,sentiment_ratio, final_summary])

# Create a DataFrame for the summarized data
summary_df = pd.DataFrame(summary_data, columns=['product_id', 'product_type', 'sentiment', 'sentiment_ratio', 'summary'])

# Display the result
summary_df

,product_id,product_type,sentiment,sentiment_ratio,summary
0,5048746,청국장,negative,12.0,"불량한 제품이 배송되었고, 교환 과정에서 문제가 발생했습니다."
1,5048746,청국장,positive,84.0,"이 제품은 맛이 진하고 구수한 맛을 좋아하는 사람들이 많아요. 집에서 간편하게 끓여 먹을 수 있어서 편리하고, 양도 푸짐해서 가족 모두에게 적합해요. 두부를 넣으면 더 맛있게 먹을 수 있고, 냄새도 적고 간도 적당하다는 의견도 있어요."


## 전체 데이터 적용 코드

In [ ]:
# sentiment별 리뷰 개수 및 전체 리뷰 개수를 계산
sentiment_counts = origin1.groupby(['product_id', 'product_type', 'sentiment']).size().reset_index(name='sentiment_count')
total_reviews = origin1.groupby(['product_id', 'product_type']).size().reset_index(name='total_reviews')

# sentiment 비율을 계산
sentiment_counts = sentiment_counts.merge(total_reviews, on=['product_id', 'product_type'])
sentiment_counts['sentiment_ratio'] = round((sentiment_counts['sentiment_count'] / sentiment_counts['total_reviews']) * 100)

# 비율을 계산한 후 리뷰 결합
grouped_reviews = origin1.groupby(['product_id', 'product_type', 'sentiment'])['cleaned_Kiwi_review'].apply(' '.join).reset_index()

# 'neutral' 감정을 제외
grouped_reviews = grouped_reviews[grouped_reviews['sentiment'] != 'neutral']

# 결합된 리뷰와 비율을 병합
grouped_data = pd.merge(grouped_reviews, sentiment_counts[['product_id', 'product_type', 'sentiment', 'sentiment_ratio']],
                      on=['product_id', 'product_type', 'sentiment'])

# 결과 출력
grouped_data


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
torch.no_grad()

In [ ]:
summary_data = []
batch_size = 2  # Specify the batch size
batch_count = 0  # Initialize a batch counter

# Loop through the grouped data (which is already filtered by a single product_id)
for _, row in grouped_data.iterrows():
    product_id = row['product_id']
    product_type = row['product_type']
    sentiment = row['sentiment']
    sentiment_ratio = row['sentiment_ratio']
    combined_reviews = row['cleaned_Kiwi_review']

    # Adjust the instruction based on sentiment
    if sentiment == 'positive':
        sentiment_instructions = "장점과 좋은 점에 집중해서 요약해주세요."
    elif sentiment == 'negative':
        sentiment_instructions = "불만 사항과 문제점에 집중해서 요약해주세요."

    # Prepare the initial summarization prompt
    summary_prompt = f"""
    당신은 상품에 대한 고객의 리뷰를 요약해서 소비자에게 정보를 제공하는 소비자 분석가입니다.

    **지시사항:**
    리뷰: {combined_reviews}\n
    이 리뷰들을 {sentiment_instructions}
    많이 나오는 내용을 중심으로 요약해주세요.
    요약한 문장을 한 번 더 매끄러운 문장으로 만들어 주세요.

    **제한사항:**
    브랜드 이름은 넣지 마세요.
    '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.

    **예시:**
    {example}

    **요약:**
    """
    inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{summary_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{combined_reviews}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    torch.cuda.empty_cache()

    # First pass: Generate the initial summary
    outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=200,
                             pad_token_id=tokenizer.eos_token_id,
                             top_p=0.90,
                             temperature=0.2
                             )
    initial_summary = extract_assistant_text(outputs)

    # Second pass: Condense the summary into one sentence
    rephrase_prompt = f"""
    당신은 문장을 간결하게 만드는 요약 전문가입니다.

    **지시사항:**
    initial summary : {initial_summary}
    위 문장을 요약해주세요.
    '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.:
    말투는 존댓말이어야 합니다.

    **요약:**
    """
    rephrase_inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{rephrase_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{initial_summary}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # Generate the final rephrased summary
    rephrase_outputs = model.generate(**tokenizer(rephrase_inputs, return_tensors="pt").to(device), max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    final_summary = extract_assistant_text(rephrase_outputs)
    torch.cuda.empty_cache()

    # Save the final summary along with sentiment
    summary_data.append([product_id, product_type, sentiment, sentiment_ratio, final_summary])
    batch_count += 1  # Increment the batch counter

    # Save to CSV every batch_size iterations
    if batch_count % batch_size == 0:
        # Create a DataFrame for the summarized data
        summary_df = pd.DataFrame(summary_data, columns=['product_id', 'product_type', 'sentiment', 'sentiment_ratio', 'summary'])

        # Append the summary_df to CSV file (creating a new file with header if it's the first batch)
        summary_df.to_csv(
            "/content/drive/MyDrive/상품별, 긍부정별 리뷰 요약.csv",
            mode='w' if batch_count == batch_size else 'a',  # 첫 배치면 새 파일 생성 (쓰기 모드), 그 이후는 추가 모드
            index=False,
            header=batch_count == batch_size,  # 첫 번째 배치에만 헤더를 추가
            encoding="utf-8-sig"
        )

        # Clear summary_data for the next batch
        summary_data = []

# Save any remaining summaries after the loop
if summary_data:
    summary_df = pd.DataFrame(summary_data, columns=['product_id', 'product_type', 'sentiment', 'sentiment_ratio', 'summary'])
    summary_df.to_csv(
        "/content/drive/MyDrive/상품별, 긍부정별 리뷰 요약.csv",
        mode='a',
        index=False,
        header=False,  # 마지막 데이터는 헤더 없이 저장
        encoding="utf-8-sig"
    )

# Display the result
summary_df


### 이전에 작업했던 내용이 있을 경우 이어서 작업하는 코드

In [ ]:
# CSV 파일에서 이미 처리된 product_id 가져오기
completed_ids_df = pd.read_csv("/content/drive/MyDrive/상품별, 긍부정별 리뷰 요약.csv", encoding="utf-8-sig")
completed_ids = completed_ids_df['product_id'].tolist()  # product_id 컬럼에서 리스트로 변환

# 처리되지 않은 product_id 필터링
grouped_data = grouped_data[~grouped_data['product_id'].isin(completed_ids)]

grouped_data

In [ ]:
import pandas as pd

summary_data = []
batch_size = 2  # Specify the batch size
batch_count = 0  # Initialize a batch counter
header_written = False  # 헤더가 이미 추가되었는지 여부를 추적

# Loop through the grouped data (which is already filtered by a single product_id)
for _, row in grouped_data.iterrows():
    product_id = row['product_id']
    product_type = row['product_type']
    sentiment = row['sentiment']
    sentiment_ratio = row['sentiment_ratio']
    combined_reviews = row['cleaned_Kiwi_review']

    # Adjust the instruction based on sentiment
    if sentiment == 'positive':
        sentiment_instructions = "장점과 좋은 점에 집중해서 요약해주세요."
    elif sentiment == 'negative':
        sentiment_instructions = "불만 사항과 문제점에 집중해서 요약해주세요."

    # Prepare the initial summarization prompt
    summary_prompt = f"""
    당신은 상품에 대한 고객의 리뷰를 요약해서 소비자에게 정보를 제공하는 소비자 분석가입니다.

    **지시사항:**
    리뷰: {combined_reviews}\n
    이 리뷰들을 {sentiment_instructions}
    많이 나오는 내용을 중심으로 요약해주세요.
    요약한 문장을 한 번 더 매끄러운 문장으로 만들어 주세요.

    **제한사항:**
    브랜드 이름은 넣지 마세요.
    '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.

    **예시:**
    {example}

    **요약:**
    """
    inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{summary_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{combined_reviews}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    torch.cuda.empty_cache()

    # First pass: Generate the initial summary
    outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=200,
                             pad_token_id=tokenizer.eos_token_id,
                             top_p=0.90,
                             temperature=0.2
                             )
    initial_summary = extract_assistant_text(outputs)

    # Second pass: Condense the summary into one sentence
    rephrase_prompt = f"""
    당신은 문장을 간결하게 만드는 요약 전문가입니다.

    **지시사항:**
    initial summary : {initial_summary}
    위 문장을 요약해주세요.
    '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.:
    말투는 존댓말이어야 합니다.

    **요약:**
    """
    rephrase_inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{rephrase_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{initial_summary}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # Generate the final rephrased summary
    rephrase_outputs = model.generate(**tokenizer(rephrase_inputs, return_tensors="pt").to(device), max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    final_summary = extract_assistant_text(rephrase_outputs)
    torch.cuda.empty_cache()

    # Save the final summary along with sentiment
    summary_data.append([product_id, product_type, sentiment, sentiment_ratio, final_summary])
    batch_count += 1  # Increment the batch counter

    # Save to CSV every batch_size iterations
    if batch_count % batch_size == 0:
        # Create a DataFrame for the summarized data
        summary_df = pd.DataFrame(summary_data, columns=['product_id', 'product_type', 'sentiment', 'sentiment_ratio', 'summary'])

        # Append the summary_df to CSV file
        summary_df.to_csv(
            "/content/drive/MyDrive/상품별, 긍부정별 리뷰 요약.csv",
            mode='a',
            index=False,
            header=not header_written,  # 첫 번째 배치에만 헤더를 추가
            encoding="utf-8-sig"
        )

        # Clear summary_data for the next batch
        summary_data = []

        # 헤더가 작성된 것으로 표시
        header_written = True

# Save any remaining summaries after the loop
if summary_data:
    summary_df = pd.DataFrame(summary_data, columns=['product_id', 'product_type', 'sentiment', 'sentiment_ratio', 'summary'])
    summary_df.to_csv(
        "/content/drive/MyDrive/상품별, 긍부정별 리뷰 요약.csv",
        mode='a',
        index=False,
        header=False,  # 마지막 데이터는 헤더 없이 저장
        encoding="utf-8-sig"
    )

# Display the result
summary_df

,product_id,product_type,sentiment,sentiment_ratio,summary
0,1000659319,이북식 백순대국,negative,4.0,"이 제품은 기대보다 맛이 떨어지는 것 같아요. 밥을 먹기 싫어 만두를 넣어 먹었는데, 맛있게 먹을 만큼은 아니었지만 삼계탕과 비슷한 맛이 나서 괜찮았습니다."
1,1000659319,이북식 백순대국,positive,94.0,"이 제품은 가격대에 비해 가성비가 좋고 맛있다는 평이 많아요. 집에서 간편하게 끓여 먹을 수 있어서 편리하고 가족 모두에게 적합하지만, 유통기한이 짧다는 단점이 있어요."
